<a href="https://colab.research.google.com/github/domschl/torch-poet/blob/master/torch_poet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import os
import shutil
from enum import Enum
import re
import time
import logging
import sys
import json
import random
import torch
import torch.nn as nn
from torch import Tensor

try:
    from urllib.request import urlopen  # Py3
except:
    print("This notebook requires Python 3.")
try:
    import pathlib
except:
    print("At least python 3.5 is needed.")
    
try: # Colab instance?
    from google.colab import drive
except: # Not? ignore.
    pass

from IPython.core.display import display, HTML

# 0. System configuration

This notebook can either run on a local jupyter server, or on google cloud.
If a GPU is available, it will be used for training (if `force_cpu` is not set to `True`).

By default snapshots of the trained net are stored locally for jupyter instances, and on user's google drive for Google Colab instances. The snapshots allow the restart of training or inference at any time, e.g. after the Colab session was terminated.

Similarily, the text corpora that are used for training, can be cached on drive or locally.

In [0]:
# force_cpu=True: use CPU for training, even if a GPU is available.
#    Note: inference uses CPU always, because that is faster.
force_cpu=False

# Define where snapshots of training data are stored:
colab_google_drive_snapshots=True

# Define if training data (the texts downloaded from internet) are cached:
colab_google_drive_data_cache=True  # In colab mode cache to google drive
local_jupyter_data_cache=True       # In local jupyter mode cache to local path

In [3]:
is_colab_notebook = 'google.colab' in sys.modules
torch_version = torch.__version__

if torch.cuda.is_available() and force_cpu is not True:
    device='cuda'
    use_cuda = True
    print(f"PyTorch {torch_version}, running on GPU")
    if is_colab_notebook:
        card = !nvidia-smi
        if len(card)>=8:
            try:
                gpu_type=card[7][6:25]
                gpu_memory=card[8][33:54]
                print(f"Colab GPU: {gpu_type}, GPU Memory: {gpu_memory}")
            except Exception as e:
                pass
else:
    device='cpu'
    use_cuda = False
    print(f"{torch_version}, running on CPU")
    if colab_notebook:
        print("Note: on Google Colab, make sure to select:")
        print("      Runtime / Change Runtime Type / Hardware accelerator: GPU")

PyTorch 1.4.0, running on GPU
Colab GPU:  Tesla P100-PCIE..., GPU Memory:      10MiB / 16280MiB


In [4]:
if is_colab_notebook:
    if colab_google_drive_snapshots:
        mountpoint='/content/drive'
        root_path='/content/drive/My Drive'
        if not os.path.exists(root_path):
            drive.mount(mountpoint)
        if not os.path.exists(root_path):
            print("Something went wrong with Google Drive access. Cannot save snapshots to GD.")
            colab_google_drive_snapshots=False
    else:
        print("Since google drive snapshots are not active, training data will be lost as soon as the Colab session terminates!")
        print("Set `colab_google_drive_snapshots` to `True` to make training data persistent.")
else:
    root_path='.'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 1. Text data collection

**Important note:** the following `project_name` determines the root directory for training data and model snapshots, so it should be changed whenever datasets of model configurations are changed.

In [0]:
project_name = "philosophers_lang_eng_ger"
project_description = "A model trained on several books of philosophers in German and English language."

In [0]:
if is_colab_notebook:
    if colab_google_drive_data_cache is True:
        data_cache_path=os.path.join(root_path,f"Colab Notebooks/{project_name}/Data")
    else:
        data_cache_path=None
else:
    if local_jupyter_data_cache is True:
        data_cache_path=os.path.join(root_path,f"{project_name}/Data")
    else:
        data_cache_path=None

if data_cache_path is not None:
    pathlib.Path(data_cache_path).mkdir(parents=True, exist_ok=True)
    if not os.path.exists(data_cache_path):
        print("ERROR, the cache directory does not exist. This will fail.")
            
def get_cache_name(cache_path, author, title):
    if cache_path is None:
        return None
    cname=f"{author} - {title}.txt"
    cname=cname.replace('?','_')  # Gutenberg index is pre-Unicode-mess and some titles contain '?' for bad conversions.
    cache_filepath=os.path.join(cache_path, cname)
    return cache_filepath

## 1.1 Project Gutenberg data source

Search, filter, clean and download books from Project Gutenberg

In [0]:
logging.basicConfig(level=logging.INFO)

In [0]:
class GutenbergLib:
    """ A fuzzy, lightweight library to access, search and filter Project Gutenberg resources """
    def __init__(self, root_url="http://www.mirrorservice.org/sites/ftp.ibiblio.org/pub/docs/books/gutenberg", cache_dir="gutenberg"):
        """ GutenbergLib by default uses a mirror's root URL
        
        root_url -- url of Project Gutenberg or any mirror URL.
        cache_dir -- path to a directory that will be used to cache the Gutenberg index and already downloaded texts
        """
        self.log = logging.getLogger('GutenbergLib')
        self.root_url = root_url
        self.index=None
        self.NEAR=2048
        try:
            if not os.path.exists(cache_dir):
                os.makedirs(cache_dir)
            self.cache_dir=cache_dir
        except Exception as e:
            self.cache_dir=None
            self.log.error(f"Failed to create cache directory {cache_dir}, {e}")

    def _parse_record(self,record,verbose=True):
        """ internal function to recreate some consistent record information from near-freestyle text """
        rl=record.split('\n')
        white=str(chr(160))+str(chr(9))+" " # non-breaking space, TAB, and space
        ebook_no=""
        while len(rl[0])>0 and rl[0][-1] in white:
            rl[0]=rl[0][:-1]
        while len(rl[0])>0 and not rl[0][-1] in white:
            ebook_no=rl[0][-1]+ebook_no
            rl[0]=rl[0][:-1]
        while len(rl[0])>0 and rl[0][-1] in white:
            rl[0]=rl[0][:-1]
        
        # Sanity check
        try:
            fa=re.findall(ebook_no,"\A[0-9]+[A-C]\Z")
        except Exception as e:
            fa=None
            if verbose is True:
                self.log.debug(f"Failed to apply regex on >{ebook_no}<")
            
        if len(rl[0])<5 or fa==None or len(ebook_no)>7:
            if verbose is True:
                print("-------------------------------------")
                print(record)
                print("- - - - - - - - - - - - - - - - - - -")
                print(f"Dodgy record: {rl[0]}")
                print(f"    ebook-id:  >{ebook_no}<")
            return None
        
        for i in range(len(rl)):
            rl[i]=rl[i].strip()
            
        p=0
        while p<len(rl)-1:
            if len(rl[p+1])==0:
                print(f"Invalid rec: {record}")
                p+=1
            else:
                if rl[p+1][0]!="[":
                    rl[p]+=" "+rl[p+1]
                    del rl[p+1]
                    if rl[p][-1]==']':
                        p+=1
                else:
                    p+=1
        
        rec={}
        l0=rl[0].split(", by ")
        rec['title']=l0[0]
        rec['ebook_id']=ebook_no
        # if len(l0)>2:
        #    print(f"Chaos title: {rl[0]}")
        if len(l0)>1:
            rec['author']=l0[-1]
        for r in rl[1:]:
            if r[0]!='[' or r[-1]!=']':
                if r[0]=='[':
                    ind=r.rfind(']')
                    if ind != -1:
                        # print(f"Garbage trail {r}")
                        r=r[:ind+1]
                        # print(f"Fixed: {r}")
                    else:
                        # print(f"Missing closing ] {r}")
                        r+=']'
                        # print(f"Fixed: {r}")
            if r[0]=='[' and r[-1]==']':
                r=r[1:-1]
                i1=r.find(':')
                if i1==-1:
                    r=r.replace("Author a.k.a.","Author a.k.a.:")
                    i1=r.find(':')
                if i1!=-1:
                    i2=r[i1:].find(' ')+i1
                else:
                    i2=-1
                if i1==-1 and i2==-1:
                    pass
                    # print(f"Invalid attribut in {rl}::{r}")
                else:
                    if i2-i1==1:
                        key=r[:i1]
                        val=r[i2+1:]
                        if '[' in key or ']' in key or '[' in val or ']' in val or len(key)>15:
                            pass
                            # print("messy key/val")
                        else:
                            rec[key.strip().lower()]=val.strip()
                    else:
                        pass
                        # print(f"Bad attribute name terminator, missing ': ' {r}")
            else:
                pass
                # print(f"Invalid attribut in {rl}::{r}")
        if len(rec)>1:
            if "language" not in rec.keys():
                rec["language"]="English"
        return rec
        
    def _parse_index(self, lines):
        """ internal function to parse the fuzzy text-based Gutenberg table of content """
        class State(Enum):
            NONE=1,
            SYNC_START=2,
            SYNC_REC=3,
            END=5
    
        white=str(chr(160))+str(chr(9))+" " # non-breaking space, TAB, and space
        state=State.NONE
        start_token="~ ~ ~ ~"
        stop_token=["====="]
        end_token="<==End"
        ignore_headers=["TITLE and AUTHOR"]
        ignore_content=["Not in the Posted Archives","human-read audio ebooks", "Audio:"]
        empty_lines=0
        records=[]
        for line in lines:
            if line[:len(end_token)]==end_token:
                state=State.END
                break

            if state==State.NONE:
                if line[:len(start_token)]==start_token:
                    state=State.SYNC_START
                    empty_lines=0
                    continue
            if state==State.SYNC_START:
                if len(line.strip())==0:
                    empty_lines+=1
                    if empty_lines>1:
                        state=State.NONE
                        continue
                else:
                    stopped=False
                    for stop in stop_token:
                        if line[:len(stop)]==stop:
                            stopped=True
                            break
                    if stopped is True:
                        state=State.NONE
                        empty_lines=0
                        continue
                    ignore=False
                    for header in ignore_headers:
                        if line[:len(header)]==header:
                            empty_lines=0
                            ignore=True
                    for token in ignore_content:
                        if token in line:
                            empty_lines=0
                            ignore=True
                    if ignore is True:
                        continue
                    rec=line
                    state=State.SYNC_REC
                    continue
            if state==State.SYNC_REC:
                if len(line.strip())==0 or line[0] not in white:
                    if len(records)<10:
                        parsed_rec=self._parse_record(rec, verbose=True)
                    else:
                        parsed_rec=self._parse_record(rec, verbose=False)
                        
                    if parsed_rec is not None:
                        records.append(parsed_rec)
                    empty_lines=1
                    if len(line.strip())==0:
                        state=State.SYNC_START
                        continue
                    else:
                        rec=line
                        continue
                rec=rec+"\n"+line
        return records
                    
    def load_index(self, cache=True, cache_expire_days=30):
        """ This function loads the Gutenberg record index, either from cache, or from a website
        
        cache -- default True, use the cache directory to cache both index and text files. Index
        expires after cache_expire_days, text files never expire. Should *NOT* be set to False
        in order to prevent unnecessary re-downloading.
        cache_expire_days -- Number of days after which the index is re-downloaded."""
        raw_index=None
        if self.cache_dir is None:
            self.log.error("Cannot cache library index, no valid cache directory.")
            return False
        ts_file=os.path.join(self.cache_dir,"timestamp")
        cache_file=os.path.join(self.cache_dir,"gutenberg_index")
        expired=True
        read_from_cache=False
        if os.path.isfile(ts_file) and os.path.isfile(cache_file):
            try:
                with open(ts_file,'r') as f:
                    ts=float(f.read())
                if time.time()-ts<cache_expire_days*24*3600:
                    expired=False
                    read_from_cache = True
                    self.log.debug("Cache timestamp read.")
                else:
                    self.log.debug("Cache for index is expired, reloading from web.")
            except:
                self.log.debug("Failed to read cache timestamp, reloading from web.")
        if expired is False and os.path.isfile(cache_file):
            try:
                with open(cache_file,'r') as f:
                    raw_index=f.read()
                    self.log.info(f"Gutenberg index read from {cache_file}")
            except:
                expired=True
                self.log.debug("Failed to read cached index, reloading from web.")
        if expired is True:
            index_url=self.root_url+"/GUTINDEX.ALL"
            try:
                raw_index = urlopen(index_url).read().decode('utf-8')
                if raw_index[0]=='\ufeff':  # Ignore BOM
                    raw_index=raw_index[1:]
                raw_index=raw_index.replace('\r','')
                self.log.info(f"Gutenberg index read from {index_url}")
            except Exception as e:
                self.log.error(f"Failed to download Gutenberg index from {index_rul}, {e}")
                return False
        if cache is True and read_from_cache is False:
            try:
                with open(ts_file,'w') as f:
                    f.write(str(time.time()))
                    self.log.debug("Wrote read cache timestamp.")
            except Exception as e:
                print(f"Failed to write cache timestamp to {ts_file}, {e}")
            try:
                with open(cache_file,'w') as f:
                    f.write(raw_index)
                    self.log.debug("Wrote read cached index.")
            except Exception as e:
                print(f"Failed to write cached index to {cache_file}, {e}")
        lines=raw_index.split('\n')
        self.records=self._parse_index(lines)
    
    def load_book(self, ebook_id):
        """ get text of an ebook from Gutenberg by ebook_id 
        
        ebook_id -- Gutenberg id
        """
        if ebook_id is None or len(ebook_id)==0:
            return None
        if ebook_id[-1]=='C':
            ebook_id=ebook_id[:-1]
        path_stub=""
        
        for i in range(len(ebook_id)-1):
            path_stub+="/"+ebook_id[i]
        path_stub+="/"+ebook_id+"/"
        filenames=[(ebook_id+"-0.txt",'utf-8'), (ebook_id+".txt",'utf-8'), (ebook_id+"-8.txt","latin1"), (ebook_id+".txt","latin1")]
        cache_name=ebook_id+".txt"
        if self.cache_dir is not None:
            cache_file=os.path.join(self.cache_dir,cache_name)
            if os.path.isfile(cache_file):
                try:
                    with open(cache_file,'r') as f:
                        data=f.read()
                        self.log.info(f"Book read from cache at {cache_file}")
                        return data
                except Exception as e:
                    self.log.error(f"Failed to read cached file {cache_file}")
        data=None
        for filename, encoding in filenames:
            file_url=self.root_url+path_stub+filename
            try:
                data = urlopen(file_url).read().decode(encoding)
                self.log.info(f"Book downloaded from {file_url}")
                break
            except Exception as e:
                self.log.debug(f"URL-Download failed: {file_url}, {e}")
                pass
        if data is None:
            self.log.warning(f"Failed to download {filenames}, last URL {file_url}, skipping book.")
            return None
        if self.cache_dir is not None:
            try:
                with open(cache_file,'w') as f:
                    f.write(data)
            except:
                self.log.error(f"Failed to cache file {cache_file}")
        return data
    
    def filter_text(self, book_text):
        """ Heuristically remove header and trailer texts not part of the actual book 
        """
        start_tokens=["*** START OF THIS PROJECT", "E-text prepared by", "This book was generously provided by the "]
        near_start_tokens=["produced by ", "Produced by ", "Transcriber's Note", "Transcriber's note:", "Anmerkungen zur Tanskription"]
        end_tokens=["End of the Project Gutenberg", "*** END OF THIS PROJECT", "***END OF THE PROJECT GUTENBER",
                   "Ende dieses Projekt Gutenberg", "End of Project Gutenberg", "Transcriber's Note"]
        blen=len(book_text)
        
        pstart=0
        for token in start_tokens:
            pos=book_text.find(token)
            if pos > pstart:
                pstart = pos
                self.log.debug(f"Start-token [{token}] found at position {pos}")
        if pstart>0:
            pos=book_text[pstart:].find("\n\n")
            if pos>=0 and pos <= self.NEAR:
                pos += pstart
                while book_text[pos]=='\n':
                    pos += 1  # eof?!
                pstart=pos
        if pstart>blen/2:
            self.log.warning("Preamble is taking more than half of the book!")
        new_book=book_text[pstart:]
        
        xpos=-1
        for token in near_start_tokens:
            pos=new_book.find(token)
            if pos>=0 and pos<=self.NEAR:
                self.log.debug(f"Near-Start-token [{token}] found at position {pos}")
                if pos>xpos:
                    xpos=pos
        if xpos > -1:
            pos2=new_book[xpos:].find("\n\n")
            self.log.debug(f"Trying extra skipping for {pos2}...")
            if pos2<=self.NEAR and pos2>0:
                self.log.debug("Trying extra skipping (2)...")
                while new_book[xpos+pos2]=='\n':
                    pos2 += 1
                new_book=new_book[xpos+pos2:]
                self.log.debug(f"Additionally shortened start by {xpos+pos2} chars")
        
        pend=len(new_book)
        for token in end_tokens:
            pos=new_book.find(token)
            if pos!=-1 and pos < pend:
                self.log.debug(f"End-token [{token}] found at pos {pos}")
                pend = pos
        if pend<len(new_book):
            pos=new_book[:pend].rfind("\n\n")
            if pos>0:
                while new_book[pos]=='\n':
                    pos -= 1  # eof?!
                pend=pos+1
        else:
            self.log.debug("No end token found!")
        if pend<len(new_book)/2:
            self.log.warning("End-text is taking more than half of the book!")
        new_book=new_book[:pend]
        return new_book
        
    def find_keywords(self,*search_keys):
        """ Search of an arbitrary number of keywords in a book record
        
        returns -- list of records that contain all keywords in any field. """
        frecs=[]
        for rec in self.records:
            found=True
            for sk in search_keys:
                subkey=False
                for key in rec.keys():
                    if sk.lower() in key.lower() or sk.lower() in rec[key].lower():
                        subkey=True
                        break
                if subkey is False:
                    found=False
                    break
            if found is True:
                frecs += [rec]
        return frecs
    
    def search(self, search_dict):
        """ Search for book record with key specific key values
        For a list of valid keys, use `get_record_keys()`
        Standard keys are:
        ebook_id, author, language, title
        example: search({"title": ["philosoph","phenomen","physic","hermeneu","logic"], "language":"english"})
        Find all books whose titles contain at least one the keywords, language english. Search keys can either be
        search for a single keyword (e.g. english), or an array of keywords. 
        returns -- list of records """
        frecs=[]
        for rec in self.records:
            found=True
            for sk in search_dict:
                if sk not in rec:
                    found=False
                    break
                else:
                    skl=search_dict[sk]
                    if not isinstance(skl,list):
                        skl=[skl]
                    nf=0
                    for skli in skl:
                        if skli.lower() in rec[sk].lower():
                            nf=nf+1
                    if nf==0:
                        found=False
                        break
            if found is True:
                frecs += [rec]
        return frecs
        
    
    def get_record_keys(self):
        """ Get a list of all keys that are used within records. Standard keys are:
        ebook_id, author, language, title
        
        returns -- list of all different keys that are somehow used."""
        rks=[]
        for r in self.records:
            rks=set(list(rks) + list(r.keys()))
        return rks

    def get_unique_record_values(self, key):
        """ Get a list of all unique values a given keys has for all records.
        get_unique_records_values('language') returns all languages in Gutenberg."""
        uv=[]
        if key not in self.get_record_keys():
            print(f"{key} is not a key used in any record!")
            return None
        for r in self.records:
            if key in r:
                uv=set(list(uv)+[r[key]])
        uv=sorted(uv)
        return uv

In [12]:
# Get the list of available books on Gutenberg.
gbl=GutenbergLib(cache_dir=os.path.join(root_path, 'gutenberg_cache'))
gbl.load_index()

INFO:GutenbergLib:Gutenberg index read from /content/drive/My Drive/gutenberg_cache/gutenberg_index


In [13]:
# sample searches
search_specs=[
    {"title": ["love", "hate", "emotion", "drama"], "language": ["english"]},
    {"author": ["brontë","Jane Austen", "Woolf", "goethe", "kant"], "language": ["english", "german"]},
    {"title": ["philosoph", "physic", "phenomen", "logic"], "language": ["english"]},
]
for search_spec in search_specs:
    book_list=gbl.search(search_spec)
    print(f"{len(book_list)} matching books found with search {search_spec}.")
# a search spec can be used by the following text library as datasource, it will automatically download, filter and prepare the content of the books requested.

305 matching books found with search {'title': ['love', 'hate', 'emotion', 'drama'], 'language': ['english']}.
105 matching books found with search {'author': ['brontë', 'Jane Austen', 'Woolf', 'goethe', 'kant'], 'language': ['english', 'german']}.
283 matching books found with search {'title': ['philosoph', 'physic', 'phenomen', 'logic'], 'language': ['english']}.


In [0]:
def create_libdesc(project_name, description, cache_path, book_list):
    libdesc={"name": project_name, "description": description, "lib": []}
    if cache_path is None or not os.path.exists(cache_path):
        print(f"A valid cache {cache_path} is needed!")
        return None
    for book_entry in book_list:
        try:
            book_raw_content=gbl.load_book(book_entry['ebook_id'])
        except Exception as e:
            print(f"Failed to download ebook_id {book_entry}, {e}")
            continue
        if book_raw_content is not None:
            try:
                book_text=gbl.filter_text(book_raw_content)
            except Exception as e:
                print(f"Internal error when filtering {book_entry}, {e}")
                continue
            filename=get_cache_name(cache_path, book_entry['author'], book_entry['title'])
            try:
                with open(filename,'w') as f:
                    f.write(book_text)
                    print(f"Cached {filename}")
                    libdesc["lib"].append((filename, book_entry['author'], book_entry['title']))
            except Exception as e:
                print(f"Failed to cache {filename}", {e})
    return libdesc

In [15]:
book_list=gbl.search({"author": ["platon", "descartes", "john locke", "david hume", "kant", "schopenhauer", "leibniz", "kierkegaard", "hegel", "nietzsche", "wittgenstein", "heidegger", "fichte"], "language": ["english", "german"]})
print(f"{len(book_list)} books found.")

83 books found.


In [0]:
# gbl.find_keywords('buddh')

In [17]:
# this will download the books! make sure it's a reasonable number of books
libdesc=create_libdesc(project_name, project_description, data_cache_path, book_list)

with open(os.path.join(data_cache_path,'libdesc.json'),'w') as f:
    json.dump(libdesc,f,indent=4)

INFO:GutenbergLib:Book read from cache at /content/drive/My Drive/gutenberg_cache/60360.txt


Cached /content/drive/My Drive/Colab Notebooks/philosophers_lang_eng_ger/Data/Friedrich Nietzsche - Der Wille zur Macht.txt


INFO:GutenbergLib:Book read from cache at /content/drive/My Drive/gutenberg_cache/60333.txt


Cached /content/drive/My Drive/Colab Notebooks/philosophers_lang_eng_ger/Data/Søren Kierkegaard - Selections from the Writings of Kierkegaard.txt


INFO:GutenbergLib:Book read from cache at /content/drive/My Drive/gutenberg_cache/60193.txt


...

Cached /content/drive/My Drive/Colab Notebooks/philosophers_lang_eng_ger/Data/Immanuel Kant - The Critique of Pure Reason.txt


INFO:GutenbergLib:Book read from cache at /content/drive/My Drive/gutenberg_cache/1998.txt


Cached /content/drive/My Drive/Colab Notebooks/philosophers_lang_eng_ger/Data/Friedrich Nietzsche - Thus Spake Zarathustra.txt


INFO:GutenbergLib:Book read from cache at /content/drive/My Drive/gutenberg_cache/59.txt


Cached /content/drive/My Drive/Colab Notebooks/philosophers_lang_eng_ger/Data/René Descartes - A Discourse on Method.txt


## 1.2 Text library

`TextLibrary` class: text library for training, encoding, batch generation,
and formatted source display. It read some books from Project Gutenberg
and supports creation of training batches. The output functions support
highlighting to allow to compare generated texts with the actual sources
to help to identify identical (memorized) parts of a given length.

In [0]:
use_dark_mode=False  # Set to false for white background

In [0]:
class TextLibrary:
    def __init__(self, descriptors, text_data_cache_directory=None, max=100000000):
        self.descriptors = descriptors
        self.data = ''
        self.cache_dir=text_data_cache_directory
        self.files = []
        self.c2i = {}
        self.i2c = {}
        index = 1
        for descriptor, author, title in descriptors:
            fd = {}
            cache_name=get_cache_name(self.cache_dir, author, title)
            if os.path.exists(cache_name):
                is_cached=True
            else:
                is_cached=False
            valid=False
            if descriptor[:4] == 'http' and is_cached is False:
                try:
                    print(f"Downloading {cache_name}")
                    dat = urlopen(descriptor).read().decode('utf-8')
                    if dat[0]=='\ufeff':  # Ignore BOM
                        dat=dat[1:]
                    dat=dat.replace('\r', '')  # get rid of pesky LFs 
                    self.data += dat
                    fd["title"] = title
                    fd["author"] = author
                    fd["data"] = dat
                    fd["index"] = index
                    index += 1
                    valid=True
                    self.files.append(fd)
                except Exception as e:
                    print(f"Can't download {descriptor}: {e}")
            else:
                fd["title"] = title
                fd["author"] = author
                try:
                    if is_cached is True:
                        print(f"Reading {cache_name} from cache")
                        f = open(cache_name)
                    else:    
                        f = open(descriptor)
                    dat = f.read(max)
                    self.data += dat
                    fd["data"] = dat
                    fd["index"] = index
                    index += 1
                    self.files.append(fd)
                    f.close()
                    valid=True
                except Exception as e:
                    print(f"ERROR: Cannot read: {filename}: {e}")
            if valid is True and is_cached is False and self.cache_dir is not None:
                try:
                    print(f"Caching {cache_name}")
                    f = open(cache_name, 'w')
                    f.write(dat)
                    f.close()
                except Exception as e:
                    print(f"ERROR: failed to save cache {cache_name}: {e}")
                
                
        ind = 0
        for c in self.data:  # sets are not deterministic
            if c not in self.c2i:
                self.c2i[c] = ind
                self.i2c[ind] = c
                ind += 1
        self.ptr = 0
        
    def display_colored_html(self, textlist, dark_mode=False, display_ref_anchor=True, pre='', post=''):
        bgcolorsWht = ['#d4e6e1', '#d8daef', '#ebdef0', '#eadbd8', '#e2d7d5', '#edebd0',
                    '#ecf3cf', '#d4efdf', '#d0ece7', '#d6eaf8', '#d4e6f1', '#d6dbdf',
                    '#f6ddcc', '#fae5d3', '#fdebd0', '#e5e8e8', '#eaeded', '#A9CCE3']
        bgcolorsDrk = ['#342621','#483a2f', '#3b4e20', '#2a3b48', '#324745', '#3d3b30',
                    '#3c235f', '#443f4f', '#403c37', '#463a28', '#443621', '#364b5f',
                    '#264d4c', '#2a3553', '#3d2b40', '#354838', '#3a3d4d', '#594C23']
        if dark_mode is False:
            bgcolors=bgcolorsWht
        else:
            bgcolors=bgcolorsDrk
        out = ''
        for txt, ind in textlist:
            txt = txt.replace('\n', '<br>')
            if ind == 0:
                out += txt
            else:
                if display_ref_anchor is True:
                    anchor="<sup>[" + str(ind) + "]</sup>"
                else:
                    anchor=""
                out += "<span style=\"background-color:"+bgcolors[ind % 16]+";\">" + \
                       txt + "</span>"+ anchor
        display(HTML(pre+out+post))

    def source_highlight(self, txt, minQuoteSize=10, dark_mode=False, display_ref_anchor=True):
        tx = txt
        out = []
        qts = []
        txsrc = [("Sources: ", 0)]
        sc = False
        noquote = ''
        while len(tx) > 0:  # search all library files for quote 'txt'
            mxQ = 0
            mxI = 0
            mxN = ''
            found = False
            for f in self.files:  # find longest quote in all texts
                p = minQuoteSize
                if p <= len(tx) and tx[:p] in f["data"]:
                    p = minQuoteSize + 1
                    while p <= len(tx) and tx[:p] in f["data"]:
                        p += 1
                    if p-1 > mxQ:
                        mxQ = p-1
                        mxI = f["index"]
                        mxN = f"{f['author']}: {f['title']}"
                        found = True
            if found:  # save longest quote for colorizing
                if len(noquote) > 0:
                    out.append((noquote, 0))
                    noquote = ''
                out.append((tx[:mxQ], mxI))
                tx = tx[mxQ:]
                if mxI not in qts:  # create a new reference, if first occurence
                    qts.append(mxI)
                    if sc:
                        txsrc.append((", ", 0))
                    sc = True
                    txsrc.append((mxN, mxI))
            else:
                noquote += tx[0]
                tx = tx[1:]
        if len(noquote) > 0:
            out.append((noquote, 0))
            noquote = ''
        self.display_colored_html(out, dark_mode=dark_mode, display_ref_anchor=display_ref_anchor)
        if len(qts) > 0:  # print references, if there is at least one source
            self.display_colored_html(txsrc, dark_mode=dark_mode, display_ref_anchor=display_ref_anchor, pre="<small><p style=\"text-align:right;\">",
                                     post="</p></small>")

    def get_slice(self, length):
        if (self.ptr + length >= len(self.data)):
            self.ptr = 0
        if self.ptr == 0:
            rst = True
        else:
            rst = False
        sl = self.data[self.ptr:self.ptr+length]
        self.ptr += length
        return sl, rst

    def decode(self, ar):
        return ''.join([self.i2c[ic] for ic in ar])

    def encode(self, s):
        return [self.c2i[c] for c in s]
        
    def get_random_slice(self, length):
        p = random.randrange(0, len(self.data)-length)
        sl = self.data[p:p+length]
        return sl

    def get_slice_array(self, length):
        ar = np.array([c for c in self.get_slice(length)[0]])
        return ar

    def get_encoded_slice(self, length):
        s, rst = self.get_slice(length)
        X = [self.c2i[c] for c in s]
        return X
        
    def get_encoded_slice_array(self, length):
        return np.array(self.get_encoded_slice(length))

    def get_sample(self, length):
        s, rst = self.get_slice(length+1)
        X = [self.c2i[c] for c in s[:-1]]
        y = [self.c2i[c] for c in s[1:]]
        return (X, y, rst)

    def get_random_sample(self, length):
        s = self.get_random_slice(length+1)
        X = [self.c2i[c] for c in s[:-1]]
        y = [self.c2i[c] for c in s[1:]]
        return (X, y)

    def get_sample_batch(self, batch_size, length):
        smpX = []
        smpy = []
        for i in range(batch_size):
            Xi, yi, rst = self.get_sample(length)
            smpX.append(Xi)
            smpy.append(yi)
        return smpX, smpy, rst

    def get_random_sample_batch(self, batch_size, length):
        smpX = []
        smpy = []
        for i in range(batch_size):
            Xi, yi = self.get_random_sample(length)
            smpX.append(Xi)
            smpy.append(yi)
        return np.array(smpX), np.array(smpy)
    
    def one_hot(p, dim):
        o=np.zeros(p.shape+(dim,), dtype=int)
        for y in range(p.shape[0]):
            for x in range(p.shape[1]):
                o[y,x,p[y,x]]=1
        return o
    
    def get_random_onehot_sample_batch(self, batch_size, length):
        X, y = self.get_random_sample_batch(batch_size, length)
        return one_hot(X,len(self.i2c)), y

## 1.3 Data sources

Data sources can either be:

1. files from local filesystem, or for colab notebooks from google drive, 
2. http(s) links

The `name` given will be use as directory name for both snapshots and model data caches.

Each entry in the `lib` array contains of:

1. (1) a local filename or (2) https(s) link
2. an Author's name
3. a title

Samples: (we are using the automatically created `libdesc` above from `GutenbergLib`), this is a manual example:
```
libdesc = {
    "name": "Women-Writers",
    "description": "A collection of works of Woolf, Austen and Brontë",
    "lib": [
        # local file:
        # ('data/tiny-shakespeare.txt', 'William Shakespeare', 'Some parts'),

        # http URLs:
        # ('http://www.mirrorservice.org/sites/ftp.ibiblio.org/pub/docs/books/gutenberg/1/0/100/100-0.txt', 'Shakespeare', 'Collected Works'),
        # ('http://www.mirrorservice.org/sites/ftp.ibiblio.org/pub/docs/books/gutenberg/3/7/4/3/37431/37431.txt', 'Jane Austen', 'Pride and Prejudice'),
        # ('http://www.mirrorservice.org/sites/ftp.ibiblio.org/pub/docs/books/gutenberg/7/6/768/768.txt', 'Emily Brontë', 'Wuthering Heights'),         
        # ('http://www.mirrorservice.org/sites/ftp.ibiblio.org/pub/docs/books/gutenberg/1/4/144/144.txt', 'Virginia Woolf', 'Voyage out'),
        # ('http://www.mirrorservice.org/sites/ftp.ibiblio.org/pub/docs/books/gutenberg/1/5/158/158.txt', 'Jane Austen', 'Emma'),
    ]
}
```

In [20]:
textlib = TextLibrary(libdesc["lib"], text_data_cache_directory=data_cache_path)

Reading /content/drive/My Drive/Colab Notebooks/philosophers_lang_eng_ger/Data/Friedrich Nietzsche - Der Wille zur Macht.txt from cache
Reading /content/drive/My Drive/Colab Notebooks/philosophers_lang_eng_ger/Data/Søren Kierkegaard - Selections from the Writings of Kierkegaard.txt from cache
Reading /content/drive/My Drive/Colab Notebooks/philosophers_lang_eng_ger/Data/Hubert Crackanthorpe - Vignettes.txt from cache
Reading /content/drive/My Drive/Colab Notebooks/philosophers_lang_eng_ger/Data/David Hume - Hume's Political Discourses.txt from cache
Reading /content/drive/My Drive/Colab Notebooks/philosophers_lang_eng_ger/Data/Georg Wilhelm Hegel - The History of Philosophy: Volume 3 of 3.txt from cache
Reading /content/drive/My Drive/Colab Notebooks/philosophers_lang_eng_ger/Data/Immanuel Kant - Die Religion innerhalb der Grenzen der bloßen Vernunft.txt from cache
Reading /content/drive/My Drive/Colab Notebooks/philosophers_lang_eng_ger/Data/Immanuel Kant - Kant's gesammelte Schriften

In [0]:
class TextLibraryDataset(torch.utils.data.Dataset):
    def __init__(self, textlib, sample_length, device, text_quanta=10):
        self.textlib=textlib
        self.device=device
        self.vocab_size=len(textlib.i2c)
        self.text_quanta=text_quanta
        self.sample_length=sample_length
        self.length=int((len(self.textlib.data)-sample_length-1)/text_quanta)
        # self.gpu_data=torch.cuda.LongTensor(textlib.encode(textlib.data[:-1]))
        self.data=torch.LongTensor(textlib.encode(textlib.data)).to(self.device)
        
    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        if idx>=self.length:
            return None
        X=self.data[idx*self.text_quanta:idx*self.text_quanta+self.sample_length].to(self.device)
        y=self.data[idx*self.text_quanta+1:idx*self.text_quanta+self.sample_length+1].to(self.device)
        return X,y

# 2. The deep LSTM model

# 2.1 Model configuration parameters

In [0]:
model_params = {
    "model_name": libdesc['name'],
    "vocab_size": len(textlib.i2c),
    "neurons": 768,
    "layers": 8,
    "learning_rate": 1e-4,
    "steps": 80,
    "batch_size": 768
}

## 2.2 The char-rnn model class

In [0]:
class Poet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, device):
        super(Poet, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.device=device
       
        self.oh = torch.eye(input_size, device=self.device)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=0)
        
        self.demb = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=-1)  # negative dims are a recent thing (as 2018-03), remove for old vers.
    
    def init_hidden(self, batch_size):
        self.h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size, device=self.device)
        self.c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size, device=self.device)

    def forward(self, inputx, steps):
        lstm_input=self.oh[inputx]
        self.lstm.flatten_parameters()
        hn, (self.h0, self.c0) = self.lstm(lstm_input, (self.h0, self.c0))
        hnr = hn.contiguous().view(-1,self.hidden_size)
        op = self.demb(hnr)
        opr = op.view(-1, steps ,self.output_size)
        return opr

    def generate(self, n, start=None, temperature=1.0):
        s=''
        torch.set_grad_enabled(False)
        if start==None or len(start)==0:
            start=' '
        self.init_hidden(1)
        for c in start:
            Xt=torch.LongTensor([[textlib.c2i[c]]])
            ypl = self.forward(Xt,1)
            ypl2 = ypl.view(-1,self.output_size)
            if temperature>0.0:
                ypl2 = ypl2 / temperature
            yp = self.softmax(ypl2)
        for i in range(n):
            ypc=Tensor.cpu(yp.detach()) # .cpu()
            y_pred=ypc.numpy()
            inds=list(range(self.output_size))
            ind = np.random.choice(inds, p=y_pred.ravel())
            s=s+textlib.i2c[ind]
            X=np.array([[ind]])
            Xt=torch.LongTensor(X)
            ypl = self.forward(Xt,1)
            ypl2 = ypl.view(-1,self.output_size)
            if temperature>0.0:
                ypl2 = ypl2 / temperature
            yp = self.softmax(ypl2)
        torch.set_grad_enabled(True)
        return s    

## 2.3 Model instance

In [0]:
poet = Poet(model_params['vocab_size'], model_params['neurons'], model_params['layers'], model_params['vocab_size'], device).to(device)

## 2.4 Optimizer

In [0]:
criterion = nn.CrossEntropyLoss()
opti = torch.optim.Adam(poet.parameters(),lr=model_params['learning_rate']);

## 2.5 Helper Functions

These allow to save or restore the training data. Saving and restoring can either be performed:

* Jupyter: store/restore in a local directory,
* Colab: store/restore on google drive. The training-code (using load_checkpoint()) will display an authentication url and code input-box in order to be able to access your google drive from this notebook. This allows to continue training sessions (or inference) after the Colab session was terminated.

In [0]:
if is_colab_notebook:
    if colab_google_drive_snapshots is True:
        snapshot_path=os.path.join(root_path,f"Colab Notebooks/{model_params['model_name']}/Snapshots")
    else:
        snapshot_path=None
else:
    snapshot_path=os.path.join(root_path,f"{model_params['model_name']}/Snapshots")

In [0]:
def get_project_path():
    if snapshot_path is None:
        return None
    project_path_ext=f"model-{model_params['vocab_size']}x{model_params['steps']}x{model_params['layers']}x{model_params['neurons']}"
    return os.path.join(snapshot_path, project_path_ext)

def create_project_path():
    if snapshot_path is None:
        return None
    ppath=get_project_path()
    pathlib.Path(ppath).mkdir(parents=True, exist_ok=True)

In [0]:
if snapshot_path is not None:
    pathlib.Path(snapshot_path).mkdir(parents=True, exist_ok=True)
    create_project_path()
    with open(os.path.join(get_project_path(),'model_params.json'),'w') as f:
        json.dump(model_params,f,indent=4)

In [0]:
def save_checkpoint(epoch, loss, pr, best_pr, filename='checkpoint.pth.tar'):
    if snapshot_path is None:
        return
    state={
            'epoch': epoch,
            'model_config': model_params,
            'state_dict': poet.state_dict(),
            'optimizer' : opti.state_dict(),
            'precision': pr,
            'loss': loss,
        }
    project_path=get_project_path()
    save_file=os.path.join(project_path,filename)
    best_file=os.path.join(project_path,'model_best.pth.tar')
    torch.save(state, save_file)
    if pr >= best_pr:
        shutil.copyfile(save_file, best_file )
        print(f"Saved best precision model, prec={pr:.3f}")
    else:
        print(f"Saved last model data, prec={pr:.3f}")

def save_history(history, filename="history.json"):
    if snapshot_path is None:
        return
    project_path=get_project_path()
    save_file=os.path.join(project_path,filename)
    try:
        with open(save_file, 'w') as f:
            json.dump(history, f)
    except Exception as e:
        print(f"Failed to write training history file {save_file}, {e}")

def load_history(filename="history.json"):
    if snapshot_path is None:
        return [], time.time()
    project_path=get_project_path()
    load_file=os.path.join(project_path,filename)
    try:
        with open(load_file, 'r') as f:
            history=json.load(f)
    except Exception as e:
        print(f"Starting new history file {load_file}")
        return [], time.time()
    if len(history)>0:
        start=history[-1]["timestamp"]
    return history, start

def load_checkpoint(filename='checkpoint.pth.tar'):
    if snapshot_path is None:
        return 0,0
    project_path=get_project_path()
    load_file=os.path.join(project_path,filename)
    if not os.path.exists(load_file):
        print(load_file)
        print("No saved state, starting from scratch.")
        return 0,0
    state=torch.load(load_file)
    mod_conf = state['model_config']
    for param in ['model_name', 'learning_rate', 'batch_size']:
        if mod_conf[param]!=model_params[param]:
            print(f"Warning: project {param} has changed from {mod_conf[param]} to {model_params[param]}")
            mod_conf[param]=model_params[param]
    if model_params!=mod_conf:
        print(f"The saved model has an incompatible configuration than the current model: {mod_conf} vs. {model_params}")
        print("Cannot restore state, starting from scratch.")
        return 0,0
    poet.load_state_dict(state['state_dict'])
    opti.load_state_dict(state['optimizer'])
    epoch = state['epoch']
    loss = state['loss']
    best_pr = state['precision']
    print(f"Continuing from saved state epoch={epoch+1}, loss={loss:.3f}")  # Save is not necessarily on epoch boundary, so that's approx.
    return epoch,loss

# 3. Training

If there is already saved training data, this step is optional, and alternatively, ch. 4 can be continued.

## 3.1 Training helpers

In [0]:
def torch_data_loader(batch_size, sample_length, device):
    textlib_dataset=TextLibraryDataset(textlib,sample_length, device)
    data_loader=torch.utils.data.DataLoader(textlib_dataset,batch_size=batch_size, shuffle=True, num_workers=0)
    return data_loader

# Get one sample:
# X, y = next(iter(data_loader))

def precision(y, yp):
    return (torch.sum(yp==y)/float((y.size()[0]*y.size()[1]))).item()

def train(Xt, yt):
    poet.zero_grad()

    poet.init_hidden(Xt.size(0))
    output = poet(Xt, model_params['steps'])
    
    olin=output.view(-1,model_params['vocab_size'])
    _, ytp=torch.max(output,2)
    ytlin=yt.view(-1)

    pr=precision(yt,ytp)
            
    loss = criterion(olin, ytlin)
    ls = loss.item()
    loss.backward()
    opti.step()

    return ls, pr

## 3.2 The actual training loop

In [0]:
ls=0
nr=0
prs=0
# torch.cuda.empty_cache()
create_project_path()
epoch_start, _ = load_checkpoint()
history, start_time = load_history()
pr=0.0
best_pr=0.0

data_loader=torch_data_loader(model_params['batch_size'], model_params['steps'], device)
    
# Make a snapshot of the trained parameters every snapshot_interval_sec
snapshot_interval_sec=180
# Generate text samples every sample_intervall_sec
sample_interval_sec=600

last_snapshot=time.time()
last_sample=time.time()

bench_all=0
bench_data=0
bench_train=0
bench_sample=0
bench_snapshot=0
bench_output_times=10  # Give 10 benchmark outputs, then stop (it stays more or less same)
sample_train_time=0

for e in range(epoch_start,2500000):
    t1=time.time()
    t0=time.time()
    for Xi,yi in data_loader:
        t2=time.time()
        # this cannot be done in data_loader, if multiprocessing is used :-/
        Xt=Xi #.to(device)
        yt=yi #.to(device)
        
        Xt.requires_grad_(False)
        yt.requires_grad_(False)

        bench_data += time.time()-t1
        t1=time.time()
        l, pr = train(Xt,yt)
        bench_train += time.time()-t1

        ls=ls+l
        prs=prs+pr
        nr=nr+1
        cur_loss=ls/nr
        cur_pr=prs/nr
        if time.time()-last_snapshot > snapshot_interval_sec:
            t1=time.time()
            nr=0
            ls=0
            prs=0
            if cur_pr>best_pr:
                best_pr=cur_pr
            last_snapshot=time.time()
            print(f"Epoch {e+1} Loss: {cur_loss:.3f} Precision: {cur_pr:.3f} Time/Sample: {sample_train_time:.6f} sec/sample")
            save_checkpoint(e,cur_loss,cur_pr, best_pr)
            # if use_cuda:
            #     print(f"Cuda memory allocated: {torch.cuda.memory_allocated()} max_alloc: {torch.cuda.max_memory_allocated()} cached: {torch.cuda.memory_cached()} max_cached: {torch.cuda.max_memory_cached()}")
            hist={"epoch": e+1, "loss": cur_loss, "precision": cur_pr, "timestamp": time.time()-start_time}
            history.append(hist)
            save_history(history)
            bench_snapshot+=time.time()-t1

            if bench_all > 0 and bench_output_times>0:
                bd=bench_data/bench_all*100.0
                bt=bench_train/bench_all*100.0
                bs=bench_sample/bench_all*100.0
                bss=bench_snapshot/bench_all*100.0
                bo=(bench_all-bench_data-bench_train-bench_sample-bench_snapshot)/bench_all*100.0
                print(f"Profiling: data-loading: {bd:.2f}%, training: {bt:.2f}%, sample gen: {bs:.2f}%, snapshots: {bss:.2f}%, overhead: {bo:.2f}%")
                bench_output_times = bench_output_times - 1
                
        sample_train_time=(time.time()-t2)/len(Xt)

        if time.time()-last_sample > sample_interval_sec and cur_loss<1.5:
            t1=time.time()
            last_sample=time.time()
            for temperature in [0.6, 0.7, 0.8]:
                print(f"Temperature {temperature}:")
                tgen=poet.generate(700,". ", temperature=temperature)
                textlib.source_highlight(tgen,minQuoteSize=10,dark_mode=use_dark_mode,display_ref_anchor=False)
            bench_sample+=time.time()-t1
        t1=time.time()
        bench_all+=time.time()-t0
        t0=time.time()


Continuing from saved state epoch=9, loss=1.070
Epoch 9 Loss: 1.070 Precision: 0.669 Time/Sample: 0.002206 sec/sample
Saved best precision model, prec=0.669
Profiling: data-loading: 0.78%, training: 100.17%, sample gen: 0.00%, snapshots: 6.30%, overhead: -7.25%
Epoch 9 Loss: 1.068 Precision: 0.670 Time/Sample: 0.002212 sec/sample
Saved best precision model, prec=0.670
Profiling: data-loading: 0.66%, training: 96.68%, sample gen: 0.00%, snapshots: 4.64%, overhead: -1.98%
Epoch 9 Loss: 1.069 Precision: 0.669 Time/Sample: 0.002205 sec/sample
Saved last model data, prec=0.669
Profiling: data-loading: 0.65%, training: 96.58%, sample gen: 0.00%, snapshots: 3.52%, overhead: -0.75%
Temperature 0.6:


Temperature 0.7:


Temperature 0.8:


Epoch 9 Loss: 1.069 Precision: 0.669 Time/Sample: 0.002185 sec/sample
Saved last model data, prec=0.669
Profiling: data-loading: 0.62%, training: 94.72%, sample gen: 2.25%, snapshots: 2.97%, overhead: -0.56%
Epoch 9 Loss: 1.067 Precision: 0.670 Time/Sample: 0.002180 sec/sample
Saved best precision model, prec=0.670
Profiling: data-loading: 0.61%, training: 95.40%, sample gen: 1.80%, snapshots: 2.89%, overhead: -0.70%
Epoch 9 Loss: 1.068 Precision: 0.670 Time/Sample: 0.002203 sec/sample
Saved last model data, prec=0.670
Profiling: data-loading: 0.62%, training: 95.63%, sample gen: 1.50%, snapshots: 2.62%, overhead: -0.37%
Temperature 0.6:


Temperature 0.7:


...

Temperature 0.8:


Epoch 17 Loss: 1.011 Precision: 0.686 Time/Sample: 0.002207 sec/sample
Saved last model data, prec=0.686
Epoch 17 Loss: 1.011 Precision: 0.686 Time/Sample: 0.002205 sec/sample
Saved last model data, prec=0.686
Epoch 17 Loss: 1.011 Precision: 0.686 Time/Sample: 0.002207 sec/sample
Saved last model data, prec=0.686
Temperature 0.6:


Temperature 0.7:


Temperature 0.8:


Epoch 17 Loss: 1.011 Precision: 0.686 Time/Sample: 0.002204 sec/sample
Saved last model data, prec=0.686
Epoch 17 Loss: 1.010 Precision: 0.687 Time/Sample: 0.002193 sec/sample
Saved last model data, prec=0.687


# 4. Text generation

## 4.1 Sample generation

In [30]:
load_checkpoint(filename="model_best.pth.tar")

Continuing from saved state epoch=17, loss=1.010


(16, 1.0101221093197459)

In [0]:
def detectPlagiarism(generatedtext, textlibrary, minQuoteLength=10, display_ref_anchor=True):
    textlibrary.source_highlight(generatedtext, minQuoteSize=minQuoteLength,dark_mode=use_dark_mode, display_ref_anchor=display_ref_anchor)

In [0]:
print("Sample text:")
print("")
for temperature in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    tgen=poet.generate(1000,"\n\n", temperature=temperature)
    print(f"================Temperature: {temperature}==============")
    detectPlagiarism(tgen, textlib, display_ref_anchor=False)

## 4.2 Dialog with the model

In [0]:
# Do a dialog with the recursive neural net trained above:
def doDialog():
    temperature = 0.6  # 0.1 (free-style chaos) - >1.0 (rigid, frozen)
    endPrompt = '.'  # the endPrompt character is the end-mark in answers.
    numSentences = 3 # Try to generate numSentences terminated by endPrompt
    # maxEndPrompts = 4  # look for number of maxEndPrompts until answer is finished.
    # maxAnswerSize = 2048  # Maximum length of the answer
    # minAnswerSize = 64  # Minimum length of the answer

    
    print("Please enter some dialog.")
    print("The net will answer according to your input.")
    print("'bye' for end,")
    print("'reset' to reset the conversation context,")
    print("'temperature=<float>' [0.1(free, chaotic) - >1.0(strict, frozen)]")
    print("    to change character of the dialog.")
    # print("    Current temperature={}.".format(temperature))
    print()
    xso = None
    bye = False
    last_ans=""
        
    while not bye:
        print("> ", end="")
        prompt = input()
        if prompt == 'bye':
            bye = True
            print("Good bye!")
            continue
        if prompt.find("temperature")>=0 and prompt.find("=") > prompt.find("temperature"):
            temperature=float(prompt[prompt.find('=')+1:])
            print(f"Temperature set to {temperature}")
            continue

        prompt+=' '
        for attempts in range(0,3):
            tgen=poet.generate(2000,prompt,temperature=temperature)
            # tgen=tgen.replace("Mr.", "Mr")
            # tgen=tgen.replace("Mrs.", "Mrs")
            # tgen=tgen.replace("\n"," ")
            # tgen=tgen.replace("  "," ")
            tgi=tgen.split(". ")
            print(f"{len(tgi)} sentences")
            if len(tgi)<numSentences:
                continue
            ans=""
            for i in range(0,numSentences):
                ans += tgi[i]+". "
            break
            # i=tgen.find(endPrompt)
            # i2=tgen[i+1:].find(endPrompt)+i
            # i3=tgen[i2+1:].find(endPrompt)+i2
            # i4=tgen[i3+1:].find(endPrompt)+i3
            # tgen=tgen[i+1:i4+2]
            # if len(tgen)>10:
            #     break
        last_ans=ans
        textlib.source_highlight(last_ans, minQuoteSize=10,dark_mode=use_dark_mode,display_ref_anchor=False)
    return

In [0]:
doDialog()

Please enter some dialog.
The net will answer according to your input.
'bye' for end,
'reset' to reset the conversation context,
'temperature=<float>' [0.1(free, chaotic) - >1.0(strict, frozen)]
    to change character of the dialog.

> Ding an sich
2 sentences
7 sentences


> A thing at itself
10 sentences


> Was ist Aufklärung?
5 sentences


> What ist the foundation of ethical action?
5 sentences


> 